# G2Net

This is my implementation of G2Net. A lot of inspiration is borrowed, I will be listing my references below. Please upvote these notebooks, you will learn a lot more from there than here!

References:
1. [[Training] G2Net Multi-Model PyTorch 💻 + W&B 🚀](https://www.kaggle.com/heyytanay/training-g2net-multi-model-pytorch-w-b)
2. [G2Net / efficientnet_b7 / baseline [inference]](https://www.kaggle.com/yasufuminakama/g2net-efficientnet-b7-baseline-inference)

# Install and Import Dependencies

In [ ]:
%%sh

pip install timm
pip install wandb --upgrade
pip install -q nnAudio

In [ ]:
import os
import platform
import wandb
from dataclasses import dataclass, field, asdict
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim

from nnAudio.Spectrogram import CQT1992v2

import timm
import albumentations as A
import albumentations.pytorch as AP

import warnings
warnings.simplefilter("ignore")

## Wandb Login

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb_key")

wandb.login(key=wandb_key)

## Configuration

In [ ]:
@dataclass
class Config:
    lr: float = 1e-5
    autocast: bool = False
    resize: tuple = (224, 224)
    model_name: str = "tf_efficientnet_b3"
    pretrained: bool = True
    epochs: int = 5
    scheduler: str = "CosineAnnealingLR"
    n_splits: int = 5
    split: float = 0.1
    folds: list = field(default_factory=lambda: [1, 2, 3, 4, 5])
    workers: int = 4
    train_bs: int = 64
    valid_bs: int = 64
    seed: int = 0
    num_labels: int = 1
    grad_acc_step: int = 1
    max_gnorm: int = 1000
    wandb: bool = True
    architecture: str = "CNN"
    competition: str = "G2Net"
    group: str = "effnet"

In [ ]:
cfg = Config()
run = wandb.init(project="g2net",
                 config=asdict(cfg),
                 group=cfg.group,
                 job_type="train"
                )

## File Paths

In [ ]:
TRAIN_PATH = "../input/g2net-gravitational-wave-detection/train"
TEST_PATH = "../input/g2net-gravitational-wave-detection/test"
TRAIN_LABELS = "../input/g2net-gravitational-wave-detection/training_labels.csv"
SAMPLE_PATH = "../input/g2net-gravitational-wave-detection/sample_submission.csv"

In [ ]:
TRAIN_FILE = "../input/g2net-gravitational-wave-detection-file-paths/training_labels_with_paths.csv"

## Helper Functions

In [ ]:
def wandb_log(**kwargs):
    """
    Logs key value pair to WandB
    """
    step = None
    if "epoch" in kwargs:
        step = kwargs["epoch"]
        del kwargs["epoch"]
    
    for k, v in kwargs.items():
        wandb.log({k: v}, step=step)
        
def get_train_file_path(image_id):
    """
    Taken from Y.Nakama's notebook
    """
    return "../input/g2net-gravitational-wave-detection/train/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id
    )

def get_test_file_path(image_id):
    """
    Taken from Y.Nakama's notebook
    """
    return "../input/g2net-gravitational-wave-detection/test/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id
    )

def convert_to_list(tensor):
    """
    Converts a tensor to list
    """
    return tensor.cpu().detach().numpy().tolist()

def quick_visual(dataset, n=5, is_test=False):
    """
    Quickly visualize dataset
    """
    for i in range(n):
        image = dataset[i]
        if not is_test:
            plt.title(f"Label: {image[1]}")
            image = image[0]
        plt.imshow(image[0])
        plt.show() 

# Define Models

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model(cfg.model_name, pretrained=cfg.pretrained, in_chans=1)
        self.n_f = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Linear(self.n_f, cfg.num_labels)
        
    def forward(self, inputs):
        return self.backbone(inputs)

# Create Dataset Class

In [ ]:
class G2NetDataset(torch.utils.data.Dataset):
    def __init__(self, data, is_test=False, transform=None):
        self.data = data
        self.is_test = is_test
        self.file_names = self.data["file_path"].values
        self.labels = self.data["target"].values
        self.wave_transform = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def apply_qtransform(self, waves, transform):
        waves = np.hstack(waves)
        waves = waves / np.max(waves)
        waves = torch.from_numpy(waves).float()
        image = transform(waves)
        return image
    
    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        waves = np.load(file_path)
        image = self.apply_qtransform(waves, self.wave_transform)
        
        if self.transform:
            image = image.squeeze().numpy()
            image = self.transform(image=image)['image']

        if self.is_test:
            return image

        label = torch.tensor(self.labels[idx]).float()        
        return image, label

# Augmentations

In [ ]:
def get_augementations(a_type="train"):
    """
    Train and Validation Augmentations
    """
    if a_type == "train":
        return A.Compose([
            AP.ToTensorV2(p=1.0),
        ], p=1.0)
    if a_type == "valid":
        return A.Compose([
            AP.ToTensorV2(p=1.0),
        ], p=1.0)

# Create Trainer

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, scheduler, train_dataloader, valid_dataloader, device):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.train_dl = train_dataloader
        self.valid_dl = valid_dataloader
        self.loss_fn = self.yield_loss
        self.valid_loss_fn = self.yield_loss
        self.device = device
        if cfg.autocast:
            self.scaler = torch.cuda.amp.GradScaler()
        
    
    def yield_loss(self, outputs, targets):
        """
        Returns the loss function
        """
        return nn.BCEWithLogitsLoss()(outputs, targets)
    
    def train_one_epoch(self):
        """
        Trains the model for one epoch
        """
        pbar = tqdm(enumerate(self.train_dl), total=len(self.train_dl))
        self.model.train()
        avg_loss = 0
        for idx, (inputs, targets) in pbar:
            image = inputs.to(self.device, dtype=torch.float)
            targets = targets.to(self.device, dtype=torch.float)
            
            if cfg.autocast:
                with torch.cuda.amp.autocast():
                    outputs = self.model(image).view(-1)
                    loss = self.loss_fn(outputs, targets)
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                outputs = self.model(image).view(-1)
                loss = self.loss_fn(outputs, targets)
                loss.backward()
                self.optimizer.step()
                
            self.optimizer.zero_grad()
            pbar.set_description(f"Loss: {loss.item():.2f}")
            
            avg_loss += loss.item()
        
        return avg_loss / len(self.train_dl)
    
    def valid_one_epoch(self):
        """
        Runs a validation epoch on the model
        """
        pbar = tqdm(enumerate(self.valid_dl), total=len(self.valid_dl))
        self.model.eval()
        
        all_targets = []
        all_preds = []
        avg_loss = 0

        with torch.no_grad():
            for idx, (inputs, targets) in pbar:
                image = inputs.to(self.device, dtype=torch.float)
                targets = targets.to(self.device, dtype=torch.float)
                
                outputs = self.model(image).view(-1)
                
                val_loss = self.valid_loss_fn(outputs, targets)
                pbar.set_description(f"Val Loss: {val_loss.item():.2f}")
                
                all_targets.extend(convert_to_list(targets))
                all_preds.extend(convert_to_list(torch.sigmoid(outputs)))
                
                avg_loss += val_loss.item()
            
            val_roc_auc = roc_auc_score(all_targets, all_preds)
            return val_roc_auc, avg_loss / len(self.valid_dl)
        
    def get_model(self):
        """
        Return model
        """
        return self.model

# Training

## Check for GPUs

In [ ]:
if torch.cuda.is_available():
    print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    DEVICE = torch.device('cuda')
else:
    print("\n[INFO] GPU not found. Using CPU: {}\n".format(platform.processor()))
    DEVICE = torch.device('cpu')

## Load and Split Data

In [ ]:
data = pd.read_csv(TRAIN_FILE)
data["file_path"] = data["id"].apply(get_train_file_path)

train_data, valid_data = train_test_split(data, test_size=cfg.split, random_state=cfg.seed)

print(f"Shape of Training Samples: {train_data.shape}")
print(f"Shape of Validation Samples: {valid_data.shape}")

In [ ]:
test_data = pd.read_csv(SAMPLE_PATH)
test_data["file_path"] = test_data["id"].apply(get_test_file_path)

print(f"Shape of Test Samples: {test_data.shape}")

## Prepare Datasets

In [ ]:
training_set = G2NetDataset(data=train_data, transform=get_augementations())
validation_set = G2NetDataset(data=valid_data)
test_set = G2NetDataset(data=test_data, is_test=True)

In [ ]:
quick_visual(training_set)

## Convert to DataLoader

In [ ]:
train_dl = torch.utils.data.DataLoader(
    training_set,
    batch_size=cfg.train_bs,
    shuffle=True,
    num_workers=cfg.workers,
    pin_memory=True
)

In [ ]:
valid_dl = torch.utils.data.DataLoader(
    validation_set,
    batch_size=cfg.valid_bs,
    shuffle=False,
    num_workers=cfg.workers,
)

## Loop

Create models folder for saving our trained models.

This will be used later to pick the model with the best score.

In [ ]:
os.mkdir("models")

In [ ]:
model = Model().to(DEVICE)
print(f"Training Model: {cfg.model_name}")

train_steps = int(len(train_data) / cfg.train_bs) * cfg.epochs

optimizer = optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=1e-6)

trainer = Trainer(model, optimizer, None, train_dl, valid_dl, DEVICE)

for epoch in tqdm(range(1, cfg.epochs + 1)):
    print(f"Epoch: {epoch} / {cfg.epochs}")
    
    train_loss = trainer.train_one_epoch()
    
    # Validate
    current_roc, valid_loss = trainer.valid_one_epoch()
    
    if cfg.wandb:
        wandb_log(
            training_loss=train_loss,
            validation_loss=valid_loss,
            roc_auc_score=current_roc,
            epoch=epoch
        )
        
    print(f"Validation ROC-AUC: {current_roc:.4f}")
    
    torch.save(trainer.get_model().state_dict(), f"models/{cfg.model_name}_{current_roc:.2f}.pt")

# Creating Submission

In [ ]:
test_dl = torch.utils.data.DataLoader(
    test_set,
    batch_size=cfg.valid_bs,
    shuffle=False,
    num_workers=cfg.workers,
)

## Inference

# Get best ROC-AUC performing model

In [ ]:
models = os.listdir("models")
sorted_list = sorted(models, key=lambda x: int(x.split("_")[-1].split(".")[1]), reverse=True)

In [ ]:
model = trainer.get_model()
model.load_state_dict(torch.load(f"models/{sorted_list[0]}"))

In [ ]:
model.eval()
pbar = tqdm(enumerate(test_dl), total=len(test_dl))
probs = []
for i, (images) in pbar:
    images = images.to(DEVICE)    
    with torch.no_grad():
        outputs = model(images).view(-1)
    probs.append(convert_to_list(torch.sigmoid(outputs)))
predictions = np.concatenate(probs)

In [ ]:
test_data['target'] = predictions
test_data[['id', 'target']].to_csv('submission.csv', index=False)
test_data.head()